In [13]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from keras.optimizers import Adam
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

from keras.callbacks import Callback

In [14]:
path = 'creditcard.csv'

df = pd.read_csv(path)
df = df.dropna()  # Supprime les lignes avec des valeurs manquantes

# Number classes
classes = df['Class'].value_counts()
X = df.iloc[:,:-1]
y = df.iloc[:,-1]  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)

In [8]:
class F1ScoreCallback(Callback):
    def __init__(self, validation_data):
        super().__init__()
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
        val_data, val_labels = self.validation_data
        y_pred = (self.model.predict(val_data) > 0.5).astype("int32")
        f1 = f1_score(val_labels, y_pred)
        logs['val_f1_score'] = f1
        print(f' — val_f1_score: {f1}')

In [15]:
#handle unbalenced data with oversampling
smote = SMOTE(random_state=42)
ros = RandomOverSampler(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
print(y_train_res.value_counts())

Class
0    227459
1    227459
Name: count, dtype: int64


In [72]:
# handle unbalenced data with undersampling 
rus = RandomUnderSampler(random_state=42)
X_train_res, y_train_res = rus.fit_resample(X_train, y_train)
print(y_train_res.value_counts())

Class
0    386
1    386
Name: count, dtype: int64


In [5]:
# handle unbalenced data with algorithm method
from keras import backend as K

def mean_false_error(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)  # Convertir y_true en float32
    # Calculer les erreurs fausses positives (FPE)
    false_positives = K.square((1 - y_true) * y_pred)
    fpe = K.sum(false_positives) / K.sum(1 - y_true)
    
    # Calculer les erreurs fausses négatives (FNE)
    false_negatives = K.square(y_true * (1 - y_pred))
    fne = K.sum(false_negatives) / K.sum(y_true)
    
    # Retourner la somme des erreurs moyennes fausses
    return fpe + fne

def mean_squared_false_error(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)  # Convertir y_true en float32
    
    # Calculer les erreurs fausses positives (FPE)
    false_positives = K.square((1 - y_true) * y_pred)
    fpe = K.sum(false_positives) / K.sum(1 - y_true)
    
    # Calculer les erreurs fausses négatives (FNE)
    false_negatives = K.square(y_true * (1 - y_pred))
    fne = K.sum(false_negatives) / K.sum(y_true)
    
    # Retourner la somme des carrés des erreurs moyennes fausses
    return K.square(fpe) + K.square(fne)

In [17]:
#Architecture Model

model = Sequential()
model.add(Dense(X.shape[1], input_dim=X.shape[1], activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['f1'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 30)                930       
                                                                 
 dense_19 (Dense)            (None, 100)               3100      
                                                                 
 dense_20 (Dense)            (None, 100)               10100     
                                                                 
 dense_21 (Dense)            (None, 100)               10100     
                                                                 
 dense_22 (Dense)            (None, 100)               10100     
                                                                 
 dense_23 (Dense)            (None, 1)                 101       
                                                                 
Total params: 34,431
Trainable params: 34,431
Non-trai

In [18]:
f1_callback = F1ScoreCallback(validation_data=(X_test, y_test))
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), callbacks=[f1_callback])

Epoch 1/10


ValueError: in user code:

    File "/opt/anaconda3/envs/cardfraud/lib/python3.9/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/opt/anaconda3/envs/cardfraud/lib/python3.9/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/anaconda3/envs/cardfraud/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/opt/anaconda3/envs/cardfraud/lib/python3.9/site-packages/keras/engine/training.py", line 1055, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "/opt/anaconda3/envs/cardfraud/lib/python3.9/site-packages/keras/engine/training.py", line 1149, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "/opt/anaconda3/envs/cardfraud/lib/python3.9/site-p

In [12]:
y_pred_test = (model.predict(X_test) > 0.5).astype("int32")
f1 = f1_score(y_test, y_pred_test)
precision = precision_score(y_test, y_pred_test)
recall = recall_score(y_test, y_pred_test)
accuracy = accuracy_score(y_test, y_pred_test)
print(f' — val_f1_score: {f1}')
print(f' — val_precision: {precision}')
print(f' — val_recall: {recall}')
print(f' — val_accuracy: {accuracy}')

1781/1781 [==============================] - 1s 361us/step
 — val_f1_score: 0.0
 — val_precision: 0.0
 — val_recall: 0.0
 — val_accuracy: 0.998139110284049


/opt/anaconda3/envs/cardfraud/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


56962 56962
